In [1]:
%%capture
from pprint import pprint

In [2]:
from LCBench.api import Benchmark
import os

os.makedirs("LCBench/cached", exist_ok=True)
bench_dir = "LCBench/cached/six_datasets_lw.json"
bench = Benchmark(bench_dir, cache=False)

==> Loading data...
==> No cached data found or cache set to False.
==> Reading json data...
==> Done.


In [3]:
import torch

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
def normalize_config(config):
    # Convert each value to a torch tensor (ensuring float type for calculations)
    batch = torch.tensor(config["batch_size"])
    lr = torch.tensor(config["learning_rate"])
    units = torch.tensor(config["max_units"])
    momentum = torch.tensor(config["momentum"])
    weight_decay = torch.tensor(config["weight_decay"])
    layers = torch.tensor(float(config["num_layers"]))
    dropout = torch.tensor(config["max_dropout"])
    
    # For log-scaled parameters: batch size, learning rate, and max units.
    batch_norm = (torch.log(batch) - torch.log(torch.tensor(16.0))) / (torch.log(torch.tensor(512.0)) - torch.log(torch.tensor(16.0)))
    lr_norm = (torch.log(lr) - torch.log(torch.tensor(1e-4))) / (torch.log(torch.tensor(1e-1)) - torch.log(torch.tensor(1e-4)))
    units_norm = (torch.log(units) - torch.log(torch.tensor(64.0))) / (torch.log(torch.tensor(1024.0)) - torch.log(torch.tensor(64.0)))
    
    # For linearly scaled parameters.
    momentum_norm = (momentum - 0.1) / (0.99 - 0.1)
    weight_decay_norm = (weight_decay - 1e-5) / (1e-1 - 1e-5)
    layers_norm = (layers - 1) / (4 - 1)
    
    # Dropout is already between 0 and 1.
    dropout_norm = dropout

    # Combine into a 7-dimensional tensor.
    normalized_vector = torch.stack([
        batch_norm, 
        lr_norm, 
        momentum_norm, 
        weight_decay_norm, 
        layers_norm, 
        units_norm, 
        dropout_norm
    ])
    
    return normalized_vector

In [ ]:
all_x = []
all_y = []
all_c = []
x2id = {}
dataset_name = "higgs"
for config_id in bench.data[dataset_name].keys():
    config = bench.query(dataset_name, "config", config_id)
    x = normalize_config(config)
    all_x.append(x)
    val_ce = bench.query(dataset_name, "final_val_cross_entropy", config_id)
    all_y.append(val_ce)
    runtime = bench.query(dataset_name, "time", config_id)[-1]
    all_c.append(runtime)
    x2id[x.numpy().tobytes()] = config_id

all_x = torch.stack(all_x)
all_y = torch.tensor(all_y).unsqueeze(1)
all_c = torch.tensor(all_c).unsqueeze(1)

In [116]:
all_x, all_y, all_c

(tensor([[0.4299, 0.4204, 0.1272,  ..., 0.6667, 0.5487, 0.0259],
         [0.9672, 0.6977, 0.0720,  ..., 1.0000, 0.9729, 0.5472],
         [0.8919, 0.1077, 0.3272,  ..., 0.0000, 0.8208, 0.3320],
         ...,
         [0.6750, 0.8598, 0.4454,  ..., 0.6667, 0.4707, 0.3635],
         [0.9691, 0.3290, 0.0093,  ..., 0.3333, 0.8684, 0.0437],
         [0.3666, 0.9906, 0.2041,  ..., 1.0000, 0.6681, 0.4045]]),
 tensor([[0.4681],
         [0.9168],
         [0.7329],
         ...,
         [0.4034],
         [0.7611],
         [0.5613]]),
 tensor([[1014.8600],
         [1367.9580],
         [ 205.0774],
         ...,
         [ 910.3070],
         [ 637.6606],
         [1356.6394]]))

In [151]:
bench.query(dataset_name, "model_parameters", x2id[all_x[0].numpy().tobytes()])

52307

In [72]:
from pandora_automl.utils import fit_gp_model
seed = 42
dim = 7
output_standardize = True
torch.manual_seed(seed)
init_config_id = torch.randint(low=0, high=2000, size=(2*(dim+1),))
config_id_history = init_config_id.tolist()
print(f"  Initial config id: {config_id_history}")
x = all_x[init_config_id]
y = all_y[init_config_id]
c = all_c[init_config_id]

model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=output_standardize)

  Initial config id: [1542, 67, 876, 414, 26, 1335, 620, 1924, 950, 1113, 1378, 1014, 1210, 954, 231, 1572]


In [73]:
posterior = model.posterior(all_x)
means = posterior.mean  # (b) x 2
vars = posterior.variance.clamp_min(1e-6)  # (b) x 2
stds = vars.sqrt()
mean_obj = means[..., 0]
std_obj = stds[..., 0]
means, vars

(tensor([[0.6447, 5.5940],
         [0.6567, 5.4204],
         [0.6523, 5.0305],
         ...,
         [0.6336, 5.4245],
         [0.6498, 5.2734],
         [0.6641, 5.6044]], grad_fn=<CloneBackward0>),
 tensor([[0.0006, 0.1223],
         [0.0006, 0.1398],
         [0.0005, 0.0986],
         ...,
         [0.0005, 0.1192],
         [0.0006, 0.1262],
         [0.0006, 0.1172]], grad_fn=<ClampMinBackward0>))

In [74]:
mgf = torch.exp(means[..., 1] + 0.5 * vars[..., 1])
mgf

tensor([285.7591, 242.3268, 160.7396,  ..., 240.8298, 207.7808, 288.0104],
       grad_fn=<ExpBackward0>)

In [52]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect

def expected_improvement_minimize(g, mu, sigma):
    z = (g - mu) / sigma
    return (g - mu) * norm.cdf(z) + sigma * norm.pdf(z)

def batch_gittins_indices(mu, sigma, cost, tol=1e-6, max_iter=100):
    mu = np.asarray(mu)
    sigma = np.asarray(sigma)
    cost = np.asarray(cost)

    l = mu - 10 * sigma
    h = mu + 10 * sigma
    m = (l + h) / 2

    for _ in range(max_iter):
        ei = expected_improvement_minimize(m, mu, sigma)
        sgn = np.sign(ei - cost)
        l = np.where(sgn <= 0, m, l)
        h = np.where(sgn >= 0, m, h)
        m = (l + h) / 2
        if np.max(np.abs(ei - cost)) < tol:
            break
    return m

def prob_f_leq_g(mu, sigma, g):
    z = (g - np.asarray(mu)) / np.asarray(sigma)
    return norm.cdf(z)

def compute_expected_cumulative_costs(gittins, probs, costs):
    idx = np.argsort(gittins)
    sorted_probs = probs[idx]
    sorted_costs = costs[idx]
    E = 0.0
    for p, c in zip(reversed(sorted_probs), reversed(sorted_costs)):
        E = c + (1 - p) * E
    return E

def cost_scaling_objective(scaling, exp_budget, means, stds, costs):
    # Scale the costs
    scaled_costs = scaling * costs
    # Compute Gittins indices with the scaled costs
    gittins = batch_gittins_indices(means, stds, scaled_costs)
    # Compute stopping probabilities
    stopping_probs = prob_f_leq_g(means, stds, gittins)
    # Compute the expected cumulative cost
    exp_cost = compute_expected_cumulative_costs(gittins, stopping_probs, costs)
    return exp_cost - exp_budget

def find_cost_scaling_custom(exp_budget, means, stds, costs, lower=0.0, upper=1.0, fixed_iters=100):
    """
    Custom bisection search that stops after a fixed number of iterations or
    when the absolute objective function value is less than ytol.

    Parameters:
      exp_budget : target expected cumulative cost.
      means, stds : arrays for the Gaussian parameters.
      costs     : array of evaluation costs.
      lower, upper : initial bracketing interval for the scaling factor.
      fixed_iters: fixed number of iterations to run.
      ytol       : tolerance on the function value f(mid).

    Returns:
      A scaling factor such that the cost_scaling_objective is nearly zero.
    """
    # First, update the upper bound to ensure f(upper) < 0.
    while cost_scaling_objective(upper, exp_budget, means, stds, costs) > 0:
        upper *= 2.0

    a, b = lower, upper
    for _ in range(fixed_iters):
        mid = (a + b) / 2.0
        f_mid = cost_scaling_objective(mid, exp_budget, means, stds, costs)
        # Check which subinterval contains the root
        if cost_scaling_objective(a, exp_budget, means, stds, costs) * f_mid < 0:
            b = mid
        else:
            a = mid

    return (a + b) / 2.0

In [75]:
exp_budget = 36000

cost_scaling = find_cost_scaling_custom(exp_budget, mean_obj.detach().numpy(), std_obj.detach().numpy(), mgf.detach().numpy())
print("Computed cost scaling:", cost_scaling)

gittins = batch_gittins_indices(mean_obj.detach().numpy(), std_obj.detach().numpy(), cost_scaling * mgf.detach().numpy())
stopping_probs = prob_f_leq_g(mean_obj.detach().numpy(), std_obj.detach().numpy(), gittins)
exp_cum_costs = compute_expected_cumulative_costs(gittins, stopping_probs, mgf.detach().numpy())
print("Expected cumulative costs:", exp_cum_costs)

Computed cost scaling: 2.029167158567526e-07
Expected cumulative costs: 36000.02671412786


In [56]:
all_x = []
all_y = []
all_c = []
dataset_name = "higgs"
for config_id in bench.data[dataset_name].keys():
    config = bench.query(dataset_name, "config", config_id)
    all_x.append(normalize_config(config))
    val_ce = bench.query(dataset_name, "final_val_cross_entropy", config_id)
    all_y.append(val_ce)
    cost = bench.query(dataset_name, "model_parameters", config_id)
    all_c.append(cost)

all_x = torch.stack(all_x)
all_y = torch.tensor(all_y).unsqueeze(1)
all_c = torch.tensor(all_c).unsqueeze(1)

In [67]:
from pandora_automl.utils import fit_gp_model
seed = 42
dim = 7
output_standardize = True
torch.manual_seed(seed)
init_config_id = torch.randint(low=0, high=2000, size=(2*(dim+1),))
config_id_history = init_config_id.tolist()
print(f"  Initial config id: {config_id_history}")
x = all_x[init_config_id]
y = all_y[init_config_id]
c = all_c[init_config_id]

model = fit_gp_model(X=x, objective_X=y, output_standardize=output_standardize)

posterior = model.posterior(all_x)
means = posterior.mean  # (b) x 2
vars = posterior.variance.clamp_min(1e-6)  # (b) x 2
stds = vars.sqrt()

means = means[..., 0].detach().numpy()
stds = stds[..., 0].detach().numpy()
costs = all_c[..., 0].detach().numpy()

  Initial config id: [1542, 67, 876, 414, 26, 1335, 620, 1924, 950, 1113, 1378, 1014, 1210, 954, 231, 1572]


In [68]:
exp_budget = 36000

cost_scaling = find_cost_scaling_custom(exp_budget, means, stds, costs)
print("Computed cost scaling:", cost_scaling)

gittins = batch_gittins_indices(means, stds, cost_scaling * costs)
stopping_probs = prob_f_leq_g(means, stds, gittins)
exp_cum_costs = compute_expected_cumulative_costs(gittins, stopping_probs, costs)
print("Expected cumulative costs:", exp_cum_costs)

Computed cost scaling: 2.010496568154024e-07
Expected cumulative costs: 35999.999999999985


In [7]:
from pandora_automl.utils import fit_gp_model
import numpy as np
import math
from botorch.acquisition import LogExpectedImprovement
from pandora_automl.acquisition.log_ei_puc import LogExpectedImprovementWithCost
from botorch.acquisition import UpperConfidenceBound
from pandora_automl.acquisition.lcb import LowerConfidenceBound
from pandora_automl.acquisition.gittins import GittinsIndex
from pandora_automl.acquisition.stable_gittins import StableGittinsIndex

In [82]:
dim = 7
n_iter = 5
maximize = False
output_standardize = True
acq = "StablePBGI(1e-5)"
seed = 5

torch.manual_seed(seed)
init_config_id = torch.randint(low=0, high=2000, size=(2*(dim+1),))
config_id_history = init_config_id.tolist()
print(f"  Initial config id: {config_id_history}")
x = all_x[init_config_id]
y = all_y[init_config_id]
c = all_c[init_config_id]
best_y_history = [y.min().item()]
best_id_history = [config_id_history[y.argmin().item()]]
cost_history = [0]

# Instead of several separate lists, we initialize a dictionary to store all acquisition histories.
acq_history = {
    'StablePBGI(1e-5)': [np.nan],
    'StablePBGI(1e-6)': [np.nan],
    'StablePBGI(1e-7)': [np.nan],
    'LogEIC-inv': [np.nan],
    'LogEIC-exp': [np.nan],
    'regret upper bound': [np.nan]
}

for i in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=output_standardize)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
        
    # 3. Define the acquisition function.
    StablePBGI_1e_5 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-5, unknown_cost=True)
    StablePBGI_1e_6 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-6, unknown_cost=True)
    StablePBGI_1e_7 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-7, unknown_cost=True)
    LogEIC_inv = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True, inverse_cost=True)
    LogEIC_exp = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True, inverse_cost=False)
    single_outcome_model = fit_gp_model(X=x, objective_X=y, output_standardize=output_standardize)
    UCB = UpperConfidenceBound(model=single_outcome_model, maximize=maximize, beta=2 * np.log(dim * ((i + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)
    LCB = LowerConfidenceBound(model=single_outcome_model, maximize=maximize, beta=2 * np.log(dim * ((i + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)

    # 4. Evaluate the acquisition function on all candidate x's.
    StablePBGI_1e_5_acq = StablePBGI_1e_5.forward(all_x.unsqueeze(1))
    StablePBGI_1e_6_acq = StablePBGI_1e_6.forward(all_x.unsqueeze(1))
    StablePBGI_1e_6_acq[config_id_history] = y.squeeze(-1)
    StablePBGI_1e_7_acq = StablePBGI_1e_7.forward(all_x.unsqueeze(1))
    LogEIC_inv_acq = LogEIC_inv.forward(all_x.unsqueeze(1))
    LogEIC_exp_acq = LogEIC_exp.forward(all_x.unsqueeze(1))
    UCB_acq = UCB.forward(all_x.unsqueeze(1))
    LCB_acq = LCB.forward(all_x.unsqueeze(1))

    # 5. Record information for stopping.
    num_configs = 2000
    all_ids = torch.arange(num_configs)
    mask = torch.ones(num_configs, dtype=torch.bool)
    mask[config_id_history] = False

    acq_history['StablePBGI(1e-5)'].append(torch.min(StablePBGI_1e_5_acq[mask]).item())
    acq_history['StablePBGI(1e-6)'].append(torch.min(StablePBGI_1e_6_acq[mask]).item())
    acq_history['StablePBGI(1e-7)'].append(torch.min(StablePBGI_1e_7_acq[mask]).item())
    acq_history['LogEIC-inv'].append(torch.max(LogEIC_inv_acq[mask]).item())
    acq_history['LogEIC-exp'].append(torch.max(LogEIC_exp_acq[mask]).item())
    acq_history['regret upper bound'].append(torch.min(UCB_acq[~mask]).item() - torch.min(LCB_acq).item())

    # 6. Select the candidate with the optimal acquisition value.
    candidate_ids = all_ids[mask]
    
    if acq == "StablePBGI(1e-5)":
        candidate_acqs = StablePBGI_1e_5_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-6)":
        candidate_acqs = StablePBGI_1e_6_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-7)":
        candidate_acqs = StablePBGI_1e_7_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "LogEIC-inv":
        candidate_acqs = LogEIC_inv_acq[mask]
        new_config_id = candidate_ids[torch.argmax(candidate_acqs)]
        new_config_acq = torch.max(candidate_acqs)
    if acq == "LogEIC-exp":
        candidate_acqs = LogEIC_exp_acq[mask]
        new_config_id = candidate_ids[torch.argmax(candidate_acqs)]
        new_config_acq = torch.max(candidate_acqs)
    if acq == "LCB":
        candidate_acqs = LCB_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)

    new_config_x = all_x[new_config_id]
    
    # 7. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 8. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    best_y_history.append(best_f.item())
    best_id_history.append(config_id_history[y.argmin().item()])
    cost_history.append(new_config_c.item())

    print(f"Iteration {i + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition value: {new_config_acq.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Cost (time): {new_config_c.item():.4f}")
    print(f"  Current best observed: {best_f.item():.4f}")
    print()

best_y_history.append(y.min().item())

  Initial config id: [1411, 814, 767, 1885, 6, 70, 1792, 233, 88, 404, 1879, 750, 1211, 1312, 318, 288]
Iteration 1:
  Selected config_id: 290
  Acquisition value: 0.6067
  Objective (final_val_cross_entropy): 0.5870
  Cost (time): 340.8998
  Current best observed: 0.5994

Iteration 2:
  Selected config_id: 1239
  Acquisition value: 0.5942
  Objective (final_val_cross_entropy): 0.6277
  Cost (time): 299.1622
  Current best observed: 0.5870

Iteration 3:
  Selected config_id: 1318
  Acquisition value: 0.5957
  Objective (final_val_cross_entropy): 0.5906
  Cost (time): 194.1711
  Current best observed: 0.5870

Iteration 4:
  Selected config_id: 1310
  Acquisition value: 0.5877
  Objective (final_val_cross_entropy): 0.5938
  Cost (time): 180.1677
  Current best observed: 0.5870

Iteration 5:
  Selected config_id: 282
  Acquisition value: 0.5931
  Objective (final_val_cross_entropy): 0.6309
  Cost (time): 201.7360
  Current best observed: 0.5870



In [ ]:
dim = 7
n_iter = 30
maximize = False
output_standardize = True
acq = "LogEIC-inv"
seed = 13

# Sample initial configurations
torch.manual_seed(seed)
init_config_id = torch.randint(low=0, high=2000, size=(2*(dim+1),))
config_id_history = init_config_id.tolist()
print(f"  Initial config id: {config_id_history}")
x = all_x[init_config_id]
y = all_y[init_config_id]
c = all_c[init_config_id]
best_y_history = [y.min().item()]
best_id_history = [config_id_history[y.argmin().item()]]
cost_history = [0]

old_model = fit_gp_model(X=x[:-1], objective_X=y[:-1], output_standardize=output_standardize)
old_config_x = x[-1]

acq_history = {
    'exp min regret gap': [np.nan],
    'delta_mu': [np.nan],
    'kappa': [np.nan],
    'kl': [np.nan],
    'ei_diff': [np.nan]
}

for i in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=output_standardize)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
        
    # 3. Define the acquisition function.
    StablePBGI_1e_5 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-5, unknown_cost=True)
    StablePBGI_1e_6 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-6, unknown_cost=True)
    StablePBGI_1e_7 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-7, unknown_cost=True)
    LogEIC_inv = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True, inverse_cost=True)
    LogEIC_exp = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True, inverse_cost=False)
    single_outcome_model = fit_gp_model(X=x, objective_X=y, output_standardize=output_standardize)
    print("sigma:", single_outcome_model.posterior(all_x.unsqueeze(1)).variance.sqrt().squeeze(-1).max().item())
    beta = 2 * np.log(dim * ((i + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5
    UCB = UpperConfidenceBound(model=single_outcome_model, maximize=maximize, beta=beta)
    LCB = LowerConfidenceBound(model=single_outcome_model, maximize=maximize, beta=beta)

    # 4. Evaluate the acquisition function on all candidate x's.
    StablePBGI_1e_5_acq = StablePBGI_1e_5.forward(all_x.unsqueeze(1))
    StablePBGI_1e_5_acq[config_id_history] = y.squeeze(-1)
    StablePBGI_1e_6_acq = StablePBGI_1e_6.forward(all_x.unsqueeze(1))
    StablePBGI_1e_6_acq[config_id_history] = y.squeeze(-1)
    StablePBGI_1e_7_acq = StablePBGI_1e_7.forward(all_x.unsqueeze(1))
    StablePBGI_1e_7_acq[config_id_history] = y.squeeze(-1)
    LogEIC_inv_acq = LogEIC_inv.forward(all_x.unsqueeze(1))
    LogEIC_exp_acq = LogEIC_exp.forward(all_x.unsqueeze(1))
    UCB_acq = UCB.forward(all_x.unsqueeze(1))
    LCB_acq = LCB.forward(all_x.unsqueeze(1))

    # 5. Select the candidate with the optimal acquisition value.
    num_configs = 2000
    all_ids = torch.arange(num_configs)
    mask = torch.ones(num_configs, dtype=torch.bool)
    mask[config_id_history] = False
    candidate_ids = all_ids[mask]
    
    if acq == "StablePBGI(1e-5)":
        candidate_acqs = StablePBGI_1e_5_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-6)":
        candidate_acqs = StablePBGI_1e_6_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-7)":
        candidate_acqs = StablePBGI_1e_7_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "LogEIC-inv":
        candidate_acqs = LogEIC_inv_acq[mask]
        new_config_id = candidate_ids[torch.argmax(candidate_acqs)]
        new_config_acq = torch.max(candidate_acqs)
    if acq == "LogEIC-exp":
        candidate_acqs = LogEIC_exp_acq[mask]
        new_config_id = candidate_ids[torch.argmax(candidate_acqs)]
        new_config_acq = torch.max(candidate_acqs)
    if acq == "LCB":
        candidate_acqs = LCB_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)

    new_config_x = all_x[new_config_id]
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]

    # 7. Record information for stopping.

    # 7.1. Get the posterior mean for old and new GPs at the new and old best points.
    # new_config_x and old_config_x should be the configurations corresponding to the current
    # and previous best indices, respectively.
    x_pair = torch.stack([new_config_x, old_config_x])
    print(new_config_x, old_config_x)

    # 7.2. Get posterior mean and covariance from the new model.
    new_posterior = single_outcome_model.posterior(x_pair)
    new_mean = new_posterior.mean         # Shape: [2]
    new_covar = new_posterior.mvn.covariance_matrix     # Shape: [2, 2]

    # 7.3. Get posterior mean and covariance from the old model.
    old_posterior = old_model.posterior(x_pair)
    old_mean = old_posterior.mean           # Shape: [2]
    old_covar = old_posterior.mvn.covariance_matrix       # Shape: [2, 2]

    # 7.4. Compute delta_mu (the absolute change in best posterior mean)
    # Here, we assume that new_config_x corresponds to the current best (new point)
    # and old_config_x corresponds to the previous best.
    delta_mu = abs(old_mean[1].item() - new_mean[0].item())
    acq_history['delta_mu'].append(delta_mu)

    # 7.5. Compute κ_{t−1} = UCB - LCB gap.
    kappa = torch.min(UCB_acq[~mask]) - torch.min(LCB_acq)
    acq_history['kappa'].append(kappa)

    # 7.6. Compute KL divergence between old and new posteriors at the new point.
    old_var = old_covar[0, 0].clamp(min=1e-12)
    new_var = new_covar[0, 0].clamp(min=1e-12)
    old_mu_val = old_mean[0]
    new_mu_val = new_mean[0]
    kl = 0.5 * (torch.log(new_var / old_var) +
                (old_var + (old_mu_val - new_mu_val).pow(2)) / new_var - 1).item()
    acq_history['kl'].append(kl)

    # 7.7. Compute ei_diff, the expected-improvement gap difference.
    # If new_config_x and old_config_x are (approximately) equal, we set ei_diff to zero.
    if not torch.allclose(new_config_x, old_config_x, atol=1e-6):
        # We use the new model's posterior for these two points.
        # new_mean and new_covar already contain the predictions.
        # Compute the difference in means:
        g = (new_mean[0] - new_mean[1]).item()
        # Compute the effective variance difference: sigma_new[0,0] - 2*sigma_new[0,1] + sigma_new[1,1]
        diff_var = (new_covar[0, 0] - 2 * new_covar[0, 1] + new_covar[1, 1]).item()
        if diff_var < 0:
            beta_val = 0.0
            pdf_val = np.sqrt(1.0 / (2 * np.pi))
            cdf_val = 1.0
        else:
            beta_val = np.sqrt(diff_var)
            u = g / beta_val if beta_val > 0 else 0.0
            pdf_val = norm.pdf(u)
            cdf_val = norm.cdf(u)
        ei_diff = beta_val * pdf_val + g * cdf_val
    else:
        ei_diff = 0.0
    acq_history['ei_diff'].append(ei_diff)

    print("delta mu:", delta_mu)
    print("kappa:", kappa.item())
    print("kl:", kl)
    print("ei diff:", ei_diff)

    # 7.8. Final expression for ΔR̃_t (the expected minimal regret gap).
    exp_min_regret_gap = delta_mu + ei_diff + kappa.item() * np.sqrt(0.5 * kl)
    print("exp min regret gap:", exp_min_regret_gap)
    print()
    acq_history['exp min regret gap'].append(exp_min_regret_gap)

    # 7.9. Reassign old_model and old_config_x for the next iteration.
    old_model = single_outcome_model
    old_config_x = new_config_x
    
    # 8. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    best_y_history.append(best_f.item())
    best_id_history.append(config_id_history[y.argmin().item()])
    cost_history.append(new_config_c.item())

    print(f"Iteration {i + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition value: {new_config_acq.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Cost (time): {new_config_c.item():.4f}")
    print(f"  Current best observed: {best_f.item():.4f}")
    print()

best_y_history.append(y.min().item())

  Initial config id: [418, 1152, 1754, 976, 1318, 1426, 1236, 329, 1964, 1498, 1667, 1626, 1460, 1806, 994, 1726]
sigma: 0.47894790294582484
tensor([0.5814, 0.6808, 0.1071, 0.3232, 0.0000, 0.3438, 0.6232]) tensor([0.9324, 0.1103, 0.5538, 0.7870, 0.6667, 0.1023, 0.3960])
delta mu: 0.3827099745140336
kappa: 0.1470150784031663
kl: 0.0015110223788405985
ei diff: 0.00010345388051661284
exp min regret gap: 0.3868543677263436

Iteration 1:
  Selected config_id: 1360
  Acquisition value: -7.5494
  Objective (final_val_cross_entropy): 0.4736
  Cost (time): 209.5318
  Current best observed: 0.3268

sigma: 0.4630851457894287
tensor([0.2496, 0.5328, 0.3502, 0.0309, 0.0000, 0.2584, 0.4609]) tensor([0.5814, 0.6808, 0.1071, 0.3232, 0.0000, 0.3438, 0.6232])
delta mu: 0.026101114573286233
kappa: 0.1913104529816585
kl: 0.006238687001181686
ei diff: 0.16316874203753642
exp min regret gap: 0.19995475264756857

Iteration 2:
  Selected config_id: 1261
  Acquisition value: -7.8306
  Objective (final_val_cros

In [164]:
def cost_function(tensor):
    costs = []
    if tensor.dim() == 1:
        tensor = tensor.unsqueeze(0)
    for x in tensor:
        model_param = bench.query(dataset_name, "model_parameters", x2id[x.numpy().tobytes()])
        costs.append(0.001*model_param)
    return torch.tensor(costs)

In [ ]:
dim = 7
n_iter = 5
maximize = False
output_standardize = True
acq = "StablePBGI(1e-5)"
seed = 5

torch.manual_seed(seed)
init_config_id = torch.randint(low=0, high=2000, size=(2*(dim+1),))
config_id_history = init_config_id.tolist()
print(f"  Initial config id: {config_id_history}")
x = all_x[init_config_id]
y = all_y[init_config_id]
c = all_c[init_config_id]
best_y_history = [y.min().item()]
best_id_history = [config_id_history[y.argmin().item()]]
cost_history = [0]

# Instead of several separate lists, we initialize a dictionary to store all acquisition histories.
acq_history = {
    'StablePBGI(1e-5)': [np.nan],
    'StablePBGI(1e-6)': [np.nan],
    'StablePBGI(1e-7)': [np.nan],
    'LogEIC': [np.nan],
    'regret upper bound': [np.nan]
}

for i in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, output_standardize=output_standardize)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
        
    # 3. Define the acquisition function.
    StablePBGI_1e_5 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-5, cost=cost_function)
    StablePBGI_1e_6 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-6, cost=cost_function)
    StablePBGI_1e_7 = StableGittinsIndex(model=model, maximize=maximize, lmbda=1e-7, cost=cost_function)
    LogEIC = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, cost=cost_function)
    UCB = UpperConfidenceBound(model=model, maximize=maximize, beta=2 * np.log(dim * ((i + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)
    LCB = LowerConfidenceBound(model=model, maximize=maximize, beta=2 * np.log(dim * ((i + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)

    # 4. Evaluate the acquisition function on all candidate x's.
    StablePBGI_1e_5_acq = StablePBGI_1e_5.forward(all_x.unsqueeze(1))
    StablePBGI_1e_6_acq = StablePBGI_1e_6.forward(all_x.unsqueeze(1))
    StablePBGI_1e_6_acq[config_id_history] = y.squeeze(-1)
    StablePBGI_1e_7_acq = StablePBGI_1e_7.forward(all_x.unsqueeze(1))
    LogEIC_acq = LogEIC.forward(all_x.unsqueeze(1))
    UCB_acq = UCB.forward(all_x.unsqueeze(1))
    LCB_acq = LCB.forward(all_x.unsqueeze(1))

    # 5. Record information for stopping.
    num_configs = 2000
    all_ids = torch.arange(num_configs)
    mask = torch.ones(num_configs, dtype=torch.bool)
    mask[config_id_history] = False

    acq_history['StablePBGI(1e-5)'].append(torch.min(StablePBGI_1e_5_acq[mask]).item())
    acq_history['StablePBGI(1e-6)'].append(torch.min(StablePBGI_1e_6_acq[mask]).item())
    acq_history['StablePBGI(1e-7)'].append(torch.min(StablePBGI_1e_7_acq[mask]).item())
    acq_history['LogEIC'].append(torch.max(LogEIC_acq[mask]).item())
    acq_history['regret upper bound'].append(torch.min(UCB_acq[~mask]).item() - torch.min(LCB_acq).item())

    # 6. Select the candidate with the optimal acquisition value.
    candidate_ids = all_ids[mask]
    
    if acq == "StablePBGI(1e-5)":
        candidate_acqs = StablePBGI_1e_5_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-6)":
        candidate_acqs = StablePBGI_1e_6_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "StablePBGI(1e-7)":
        candidate_acqs = StablePBGI_1e_7_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)
    if acq == "LogEIC":
        candidate_acqs = LogEIC_acq[mask]
        new_config_id = candidate_ids[torch.argmax(candidate_acqs)]
        new_config_acq = torch.max(candidate_acqs)
    if acq == "LCB":
        candidate_acqs = LCB_acq[mask]
        new_config_id = candidate_ids[torch.argmin(candidate_acqs)]
        new_config_acq = torch.min(candidate_acqs)

    new_config_x = all_x[new_config_id]
    
    # 7. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 8. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    best_y_history.append(best_f.item())
    best_id_history.append(config_id_history[y.argmin().item()])
    cost_history.append(new_config_c.item())

    print(f"Iteration {i + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition value: {new_config_acq.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Cost (time): {new_config_c.item():.4f}")
    print(f"  Current best observed: {best_f.item():.4f}")
    print()

best_y_history.append(y.min().item())

  Initial config id: [1411, 814, 767, 1885, 6, 70, 1792, 233, 88, 404, 1879, 750, 1211, 1312, 318, 288]
Iteration 1:
  Selected config_id: 936
  Acquisition value: 0.5482
  Objective (final_val_cross_entropy): 0.6418
  Cost (time): 143.8312
  Current best observed: 0.5994

Iteration 2:
  Selected config_id: 231
  Acquisition value: 0.5491
  Objective (final_val_cross_entropy): 0.6447
  Cost (time): 168.3126
  Current best observed: 0.5994

Iteration 3:
  Selected config_id: 860
  Acquisition value: 0.5521
  Objective (final_val_cross_entropy): 0.6450
  Cost (time): 132.8949
  Current best observed: 0.5994

Iteration 4:
  Selected config_id: 285
  Acquisition value: 0.5560
  Objective (final_val_cross_entropy): 0.6445
  Cost (time): 130.9145
  Current best observed: 0.5994

Iteration 5:
  Selected config_id: 1332
  Acquisition value: 0.5585
  Objective (final_val_cross_entropy): 0.6418
  Cost (time): 122.5977
  Current best observed: 0.5994

